In [1]:
from typing import List, Sequence, Annotated,TypedDict # for type hints
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage # for message types
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from chains import generation_chain, reflection_chain
load_dotenv()

True

In [2]:
#!pip install langgraph
# !pip install nbimporter
# !pip install nbformat
# state definition
class State(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
# state graph
graph=StateGraph(State)
# for better flexibility MessageGraph is deprecated and StateGraph became standard

In [3]:
REFLECT = 'reflect'
GENERATE = 'generate'

In [4]:
# two distinct, independent 'pipelines' for your agent.
def generate_node(state):
    return generation_chain.invoke({
        'messages':state
    })

def reflect_node(state):
    response = reflection_chain.invoke({
        'messages':state
    })
    return [HumanMessage(content=response.content)] # tricking the system that the reflection is being done by an human

In [5]:
graph.add_node(GENERATE, generate_node)
graph.add_node(REFLECT, reflect_node)

graph.set_entry_point(GENERATE)

In [6]:
def should_continue(state):
    if(len(state)>4):
        return END
    return REFLECT

In [7]:
# Adding edges or connecting nodes
# conditional or relfection edge
graph.add_conditional_edges(GENERATE, should_continue) 
graph.add_edge(REFLECT, GENERATE)

In [8]:
! pip install grandalf
app = graph.compile()

print(app.get_graph().draw_mermaid())
app.get_graph().print_ascii()

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__(<p>__start__</p>)
	generate(generate)
	reflect(reflect)
	__end__(<p>__end__</p>)
	__start__ --> generate;
	generate --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
+----------+   
| generate |   
+----------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [ ]:
# response = app.invoke([HumanMessage(content="AI AGENTS TAKING OVER CONTENT CREATION")])
# print(response) this will not work, as i'm using stategraph and it expects dict here
response = app.invoke(
    {"messages": [HumanMessage(content="AI AGENTS TAKING OVER CONTENT CREATION")]}
)

print(response)

ValueError: variable messages should be a list of base messages, got {'messages': [HumanMessage(content='AI AGENTS TAKING OVER CONTENT CREATION', additional_kwargs={}, response_metadata={}, id='a913b1c0-a448-4345-815b-6a9c506f7645')]} of type <class 'dict'>